In [1]:
# Importing Libraries
import ast
import pandas as pd
from datasets import load_dataset
import matplotlib.pyplot as plt 
import seaborn as sns

# Loading Data
dataset = load_dataset('lukebarousse/data_jobs')
df = dataset['train'].to_pandas()

# Data Cleanup
df['job_posted_date'] = pd.to_datetime(df['job_posted_date'])
df['job_skills'] = df['job_skills'].apply(lambda x: ast.literal_eval(x) if pd.notna(x) else x)

In [2]:
df.head(3)

,job_title_short,job_title,job_location,job_via,job_schedule_type,job_work_from_home,search_location,job_posted_date,job_no_degree_mention,job_health_insurance,job_country,salary_rate,salary_year_avg,salary_hour_avg,company_name,job_skills,job_type_skills
0,Senior Data Engineer,Senior Clinical Data Engineer / Principal Clin...,"Watertown, CT",via Work Nearby,Full-time,False,"Texas, United States",2023-06-16 13:44:15,False,False,United States,None,NaN,NaN,Boehringer Ingelheim,None,None
1,Data Analyst,Data Analyst,"Guadalajara, Jalisco, Mexico",via BeBee México,Full-time,False,Mexico,2023-01-14 13:18:07,False,False,Mexico,None,NaN,NaN,Hewlett Packard Enterprise,"[r, python, sql, nosql, power bi, tableau]","{'analyst_tools': ['power bi', 'tableau'], 'pr..."
2,Data Engineer,"Data Engineer/Scientist/Analyst, Mid or Senior...","Berlin, Germany",via LinkedIn,Full-time,False,Germany,2023-10-10 13:14:55,False,False,Germany,None,NaN,NaN,ALPHA Augmented Services,"[python, sql, c#, azure, airflow, dax, docker,...","{'analyst_tools': ['dax'], 'cloud': ['azure'],..."


In [ ]:
df_US = df[df['job_country']=='United States'].copy()
df_skills = df_US.explode('job_skills')
df_skills[['job_title','job_skills']]

In [70]:
df_skills_count = df_skills.groupby(['job_title_short','job_skills']).size().reset_index(name='skill_count')
df_skills_count.sort_values(by='skill_count',ascending=False,inplace=True)
df_skills_count.head(10)

,job_title_short,job_skills,skill_count
886,Data Scientist,python,42379
472,Data Analyst,sql,34452
921,Data Scientist,sql,30034
351,Data Analyst,excel,27519
890,Data Scientist,r,26022
694,Data Engineer,sql,23958
659,Data Engineer,python,22762
483,Data Analyst,tableau,19311
438,Data Analyst,python,18382
530,Data Engineer,aws,15018


In [ ]:
# job_count = df['job_title_short'].value_counts().sort_values(ascending=False).reset_index(name='job_count').head(3)
job_titles = df_skills_count['job_title_short'].unique().tolist()
job_titles[0:3]

In [ ]:
job_count = df['job_title_short'].value_counts().sort_values(ascending=False).reset_index(name='job_count').head(3)
job_titles = job_count['job_title_short'].to_list()
# len(job_titles)

sns.set_theme(style='ticks')
fig, ax = plt.subplots(len(job_titles),1)   #,figsize=(12,5)

for i,job in enumerate(job_titles):
    df_filter = df_skills_count[df_skills_count['job_title_short']==job].head()
    sns.barplot(data=df_filter,x='skill_count',y='job_skills',ax=ax[i],hue='job_skills',palette='dark:b',legend=False)  
    # sns.despine()
    ax[i].set_title(f'{job}')
    ax[i].set_xlabel('')
    ax[i].set_ylabel('')
    ax[i].set_xlim(0,45000)

fig.suptitle('Counts of Skills Requested in US Job Postings', fontsize=15)
fig.tight_layout()
plt.show()

In [ ]:
df_job_title_count = df['job_title_short'].value_counts(ascending=False).reset_index(name='job_total')
df_job_title_count

,job_title_short,job_count
0,Data Analyst,196075
1,Data Engineer,186241
2,Data Scientist,172286
3,Business Analyst,49063
4,Software Engineer,44929
5,Senior Data Engineer,44563
6,Senior Data Scientist,36957
7,Senior Data Analyst,29216
8,Machine Learning Engineer,14080
9,Cloud Engineer,12331
